In [1]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from collections import defaultdict
from scipy import spatial
from IPython.display import HTML, display
import tabulate
import json
import pickle
import re
import nltk

# Part 1: Feature Preparation

# Dataset Prep

### 1. Test set 

In [2]:
df = pd.read_excel("InputTestSet-Reviews48_Ann_NEW.xlsx")

In [3]:
df.head()

,UID,PID,Dec,Sent,MComp,Cat,SubCat
0,0,2019_SJf_XhCqKm,Reject,The authors propose to use k-DPP to select a s...,0,NaN,NaN
1,1,2019_SJf_XhCqKm,Reject,"This paper covers the related work nicely, wit...",0,NaN,NaN
2,2,2019_SJf_XhCqKm,Reject,The rest of the paper are also clearly written.,0,NaN,NaN
3,3,2019_SJf_XhCqKm,Reject,"However, I have some concerns about the propos...",0,NaN,NaN
4,4,2019_SJf_XhCqKm,Reject,"- It is not clear how to define the kernel, th...",0,NaN,NaN


In [4]:
df.shape

(1505, 7)

In [5]:
gt_dict = {}

for i in range(0, df.shape[0]):
    pid = df.loc[i]["PID"]
    if not pid in gt_dict:
        gt_dict[pid] = {"dec": df.loc[i]["Dec"], "mcomp": set(), "not_mcomp": set()}
    if df.loc[i]["MComp"] == 1:
        gt_dict[pid]["mcomp"].add(df.loc[i]["UID"])
    else:
        gt_dict[pid]["not_mcomp"].add(df.loc[i]["UID"])

In [6]:
stats_dict = {"Accept": [0, 0], "Reject": [0, 0]}

for k, v in gt_dict.items():
    #print(len(v["mcomp"]), len(v["not_mcomp"]), v["dec"])
    stats_dict[v["dec"]][0] += len(v["mcomp"])
    stats_dict[v["dec"]][1] += len(v["not_mcomp"])
    
print(stats_dict)

{'Accept': [49, 643], 'Reject': [68, 745]}


In [7]:
test_set = list(gt_dict.keys())
print("TestSet length: %d\n"%len(test_set), test_set)

TestSet length: 32
 ['2019_SJf_XhCqKm', '2017_Bk0MRI5lg', '2020_SyevYxHtDB', '2018_rJBiunlAW', '2020_rkltE0VKwH', '2018_Hki-ZlbA-', '2019_BJx0sjC5FX', '2020_r1e_FpNFDr', '2020_B1lsXREYvr', '2018_SkZxCk-0Z', '2019_rJzoujRct7', '2018_HkfXMz-Ab', '2017_BJ9fZNqle', '2019_SyxZJn05YX', '2017_B1ckMDqlg', '2017_HJ0NvFzxl', '2017_S1_pAu9xl', '2018_SyYYPdg0-', '2017_BJAA4wKxg', '2019_HyVxPsC9tm', '2019_HylTBhA5tQ', '2019_B1l08oAct7', '2018_H135uzZ0-', '2017_H1oyRlYgg', '2017_r1y1aawlg', '2020_r1eX1yrKwB', '2020_Byg79h4tvB', '2019_H1lFZnR5YX', '2020_BkeWw6VFwr', '2018_HyHmGyZCZ', '2018_HyUNwulC-', '2020_HkgsPhNYPS']


In [8]:
for k in test_set:
    print('{:20}{}'.format(k, gt_dict[k]["mcomp"]))

2019_SJf_XhCqKm     {39, 17, 27, 28, 30}
2017_Bk0MRI5lg      {57}
2020_SyevYxHtDB     {76, 87}
2018_rJBiunlAW      {108, 110, 112, 113, 124, 126}
2020_rkltE0VKwH     {160, 184, 155, 157, 159}
2018_Hki-ZlbA-      {267, 235, 236, 271}
2019_BJx0sjC5FX     {292, 287}
2020_r1e_FpNFDr     {312, 322, 315, 308}
2020_B1lsXREYvr     {376, 401}
2018_SkZxCk-0Z      {449, 443, 445, 486}
2019_rJzoujRct7     {518, 519}
2018_HkfXMz-Ab      {573, 566}
2017_BJ9fZNqle      {627, 623, 615}
2019_SyxZJn05YX     {672, 673, 657, 669, 671}
2017_B1ckMDqlg      {714, 707}
2017_HJ0NvFzxl      {739}
2017_S1_pAu9xl      {806, 808, 809, 810, 792}
2018_SyYYPdg0-      {834, 867, 868, 869, 870, 872, 873, 844, 830}
2017_BJAA4wKxg      {884}
2019_HyVxPsC9tm     {931, 933, 905, 909, 912, 913, 919, 926}
2019_HylTBhA5tQ     {972, 950}
2019_B1l08oAct7     {994, 996, 1064, 1004, 1007, 1044, 1047, 1048, 1055}
2018_H135uzZ0-      {1072, 1079}
2017_H1oyRlYgg      set()
2017_r1y1aawlg      {1125, 1162, 1100, 1102, 1168}
2020_r1eX

In [9]:
sents_for_test = defaultdict(list)

for i in range(0, df.shape[0]):
    pid = df.loc[i]["PID"]
    test_sent_raw = str(df.loc[i]["Sent"])
    
    # Replace URLs with [URL]
    test_sent_raw = re.sub(r'http[s]?://[a-zA-z\.\-/0-9~]*', "[URL]", test_sent_raw)
    test_sent_raw = re.sub(r'papers.nips.cc/paper/[a-zA-z\.\-/0-9~]*', "[URL]", test_sent_raw)
    test_sent_raw = re.sub(r'arxiv.org/[a-zA-z\.\-/0-9~]*', "[URL]", test_sent_raw)
    
    sents_for_test[pid].append((df.loc[i]["UID"], test_sent_raw))

### 2. Train Set

In [10]:
df_train = pd.read_excel("InputTrainSet-Reviews7_Ann.xlsx")
df_train.head()

,UID,PID,Dec,Sent,MComp,Cat,SubCat
0,243,2020_ryen_CEFwr,Reject,It extends this approach by introducing an add...,0,None,None
1,179,2018_H1LAqMbRW,Reject,"Experimentally, the results are rather weak co...",0,None,None
2,157,2017_HyTqHL5xg,Accept,The experiments are interesting but I'm still ...,0,None,None
3,146,2017_HyTqHL5xg,Accept,Section 2.2 says they do the latter in the int...,0,None,None
4,90,2017_ByToKu9ll,Reject,4)This paper proposed an improved version of t...,0,None,None


In [11]:
train_sets = {"mcomp": [], "non_mcomp": []}

In [12]:
for i in range(0, df_train.shape[0]):
    pid = df_train.loc[i]["PID"]
    train_sent_raw = str(df_train.loc[i]["Sent"])
    
    type_comp = df_train.loc[i]["MComp"]
    
    if type_comp == 1:
        train_sets["mcomp"].append(train_sent_raw)
    else:
        train_sets["non_mcomp"].append(train_sent_raw)

In [13]:
len(train_sets["mcomp"]), len(train_sets["non_mcomp"])

(26, 270)

## Load entities

In [14]:
with open("entities_dict_smaller", "r") as f:
    entity_dict = json.load(f)

In [15]:
set(entity_dict.values())

{'Material', 'Method', 'Metric', 'Task'}

In [16]:
list(entity_dict.items())[0:20]

[('convolutional neural networks', 'Method'),
 ('convnets', 'Method'),
 ('recognition', 'Task'),
 ('visual recognition tasks', 'Task'),
 ('age estimation', 'Task'),
 ('head pose estimation', 'Task'),
 ('multi - label classification', 'Task'),
 ('semantic segmentation', 'Task'),
 ('classification', 'Task'),
 ('deep convnets', 'Method'),
 ('dldl', 'Method'),
 ('feature learning', 'Task'),
 ('deep learning', 'Method'),
 ('image classification', 'Task'),
 ('deep learning methods', 'Method'),
 ('image classification tasks', 'Task'),
 ('human pose estimation', 'Task'),
 ('convnet', 'Method'),
 ('recognition tasks', 'Task'),
 ('ensemble', 'Method')]

In [17]:
entity_key_map = {}
for i in entity_dict:
    s = re.sub('[^0-9a-zA-Z,:;.?!\- ]+', '', i)
    while s.find("  ") > -1:
        s = s.replace("  ", " ")
    if len(s) > 2:
        cl = re.sub('[^0-9a-zA-Z ]+', '', i)
        while cl.find("  ") > -1:
            cl = cl.replace("  ", " ")
        entity_key_map[cl.strip()] = i
print(len(entity_key_map))

1784


In [18]:
coun = 0
for i in entity_dict:
    if len(i) < 5:
        coun +=1
#         print(i)
print(coun)

212


In [19]:
list(entity_key_map.items())[0:5]

[('convolutional neural networks', 'convolutional neural networks'),
 ('convnets', 'convnets'),
 ('recognition', 'recognition'),
 ('visual recognition tasks', 'visual recognition tasks'),
 ('age estimation', 'age estimation')]

In [20]:
from collections import Counter
c = Counter(entity_dict.values())
c

Counter({'Material': 165, 'Method': 1191, 'Metric': 158, 'Task': 289})

In [21]:
# dir(c)
reverse_map = defaultdict(list)

for k, v in entity_dict.items():
    reverse_map[v].append(k)

In [22]:
# for i in reverse_map["Task"]:
#     print(i)

In [23]:
"MNIST" in entity_key_map, "mnist" in entity_key_map

(False, True)

## A. RoBERTa trained on SciLit

In [14]:
import spacy
import torch

In [16]:
!pip3 list | grep -E 'transformers|tokenizers'

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
tokenizers (0.7.0)
transformers (2.10.0)


In [17]:
from transformers import AutoTokenizer, AutoModel

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [18]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")

Widget Javascript not detected.  It may not be installed or enabled properly.


In [19]:
import transformers
print(transformers.__version__)

2.10.0


In [20]:
def embed_text_using_roberta(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    return last_hidden_states

In [52]:
# def mask_entities(sentence, replace_with_dataset=True):
#     cleaned_sent = re.sub('[^0-9a-zA-Z,:;.?!\- ]+', ' ', sentence)
#     while cleaned_sent.find("  ") > -1:
#         cleaned_sent = cleaned_sent.replace("  ", " ")
    
#     entity_key_map_keys = list(entity_key_map.keys()) # As we will be dunamically adding entries to this dict an dthat will throw an error.
#     entities_found = []
#     for i in entity_key_map_keys:
#         if cleaned_sent.find(" " + i + " ") > -1:
#             entities_found.append(i)
#         elif cleaned_sent.lower().find(" " + i + " ") > -1:
#             found_idx = cleaned_sent.lower().find(" " + i + " ")
#             entity_dict[cleaned_sent[found_idx:found_idx+len(" " + i + " ")]] = entity_dict[i]
#             entity_key_map[cleaned_sent[found_idx:found_idx+len(" " + i + " ")]] = entity_key_map[i]
    
#     entities_found.sort(key=lambda s: len(s))
#     len_sorted_entities = entities_found.copy()
    
#     subset_entities = []
#     # Remove subset entities (eg: Among cnn and 3-layer-cnn, prefer the latter)
#     for fe in len_sorted_entities:
#         for other_ent in len_sorted_entities:
#             if fe != other_ent and other_ent.find(fe) > -1:
#                 subset_entities.append(fe)
#                 break
#     for se in subset_entities:
#         len_sorted_entities.remove(se)
#     for maxents in len_sorted_entities:
#         mask_name = " " + entity_dict[entity_key_map[i]].lower() + " "
#         if replace_with_dataset:
#             if mask_name == " material ":
#                 mask_name = " dataset "
#         cleaned_sent = cleaned_sent.replace(" " + maxents + " ", mask_name)
#     words_cleaned = nltk.word_tokenize(cleaned_sent)
#     dups_removed = [v for i, v in enumerate(words_cleaned) if i == 0 or v != words_cleaned[i-1]]
#     new_dup_removed_sent = " ".join(dups_removed)
#     return new_dup_removed_sent.strip()

# #     #print(cleaned_sent)
# #     for i in entity_key_map:
# #         if cleaned_sent.find(" " + i + " ") > -1:
# #             #print("Substituting ent: {} with mask: {}".format(i, entity_dict[entity_key_map[i]].lower()))
# #             cleaned_sent = cleaned_sent.replace(i, entity_dict[entity_key_map[i]].lower())
# #     return cleaned_sent

In [21]:
nlp = spacy.load('en_core_web_sm')
sp_toks = ["result", "method", "task", "dataset", "metric", "baseline", "fair", "unfair"]

In [23]:
def extract_chunks_using_spacy_dp(conssentence, replace_with_dataset=True):
    return []
# #     conssentence = mask_entities(conssentence, replace_with_dataset)
# #     print(conssentence)
#     doc = nlp(conssentence)
#     verb_subtree = []

#     for s in doc.sents:
# #         find_special_tokens = {"compar": [], "result": [], "method": [], "technique": [], "task": [], "dataset": [], "material": [], "metric": []}
#         find_special_tokens = {"compar": [], "result": [], "method": [], "baseline": [], "task": [], 
#                                "dataset": [],  "metric": [], "unfair": [], "fair": []}

#         for tok in s:

#             if tok.text.lower().startswith("compar"):
#                 find_special_tokens["compar"].append(tok)
#             else:
#                 for k in sp_toks:
#                     if tok.text.lower().startswith(k):
#                         find_special_tokens[k].append(tok)
#                         break

#         verb_tokens = []
#         if find_special_tokens["compar"]:
#             for t in find_special_tokens["compar"]:
# #                     verb_subtree.append(t.subtree)
#                 if t == s.root:
#                     simplified_sent = ""
#                     for chh in t.lefts:
#                         simplified_sent = simplified_sent + " " + chh.text
#                     simplified_sent = simplified_sent + " " + t.text
#                     for chh in t.rights:
#                         simplified_sent = simplified_sent + " " + chh.text
# #                         print("SIMP: ", simplified_sent)
#                     verb_subtree.append(simplified_sent)
#                 else:
#                     verb_subtree.append(t.subtree)
#         else:
#             for k in sp_toks:
#                 for i in find_special_tokens[k]:
#                     local_vt = []
#                     for j in i.ancestors:
#                         if j.pos_ == "NOUN":
#                             local_vt.append(j)
#                     if not local_vt:
#                         for j in i.ancestors:
#                             if j.pos_ == "VERB":
#                                 local_vt.append(j)
#                     verb_tokens = verb_tokens + local_vt


#             for i in verb_tokens:
#                 verb_subtree.append(i.subtree)

#     eecc = []
#     for i in verb_subtree:
#         if type(i) == str:
#             eecc.append(i)
#         else:
#             local_chunk = ""
#             for lcaltok in i:
#                 local_chunk = local_chunk + " " + lcaltok.text
#             eecc.append(local_chunk)
# #     if not eecc:
# #         print(conssentence)
#     return list(set(eecc))


### Computing vectors of the initial training pool of sentences

In [24]:
train_pool_roberta_vecs = {"mcomp": [], "non_mcomp": []}
single_train_pool_roberta_vecs = {"mcomp": [], "non_mcomp": []}
train_pool_uid_vecs = defaultdict(list)
mc_nmc_fake = {}

In [25]:
fake_idx = 0

for i in train_sets["mcomp"]:
    mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(i)
    if mcomp_chunks_from_sent:
        final_chunks = mcomp_chunks_from_sent
    else:
        final_chunks = [i]
    
    mc_nmc_fake[fake_idx] = 1
    for single_chunk in final_chunks:
        vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
        train_pool_uid_vecs[fake_idx].append(vec / norm(vec))
        train_pool_roberta_vecs["mcomp"].append(vec/norm(vec))
    
    collated_chunk = " ".join(final_chunks)
    vec = embed_text_using_roberta(collated_chunk.strip()).mean(1).detach().numpy()
    single_train_pool_roberta_vecs["mcomp"].append(vec/norm(vec))
    fake_idx += 1


for i in train_sets["non_mcomp"]:
    mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(i)
    if mcomp_chunks_from_sent:
        final_chunks = mcomp_chunks_from_sent
    else:
        final_chunks = [i]
    
    mc_nmc_fake[fake_idx] = 0
    for single_chunk in final_chunks:
        vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
        train_pool_uid_vecs[fake_idx].append(vec / norm(vec))
        train_pool_roberta_vecs["non_mcomp"].append(vec/norm(vec))
    
    collated_chunk = " ".join(final_chunks)
    vec = embed_text_using_roberta(collated_chunk.strip()).mean(1).detach().numpy()
    single_train_pool_roberta_vecs["non_mcomp"].append(vec/norm(vec))
    fake_idx +=1

In [26]:
roberta_vectors = defaultdict(dict)

skip_uids = []

for pid in gt_dict:
    roberta_vectors[pid] = {}
    
    for mcs in gt_dict[pid]["mcomp"]:
        try:
            mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(df.loc[mcs]["Sent"])
            if mcomp_chunks_from_sent:
                final_chunks = mcomp_chunks_from_sent
            else:
                final_chunks = [df.loc[mcs]["Sent"]]
            
            roberta_vectors[pid][mcs] = []
            for single_chunk in final_chunks:
                vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
                roberta_vectors[pid][mcs].append(vec / norm(vec))
        except Exception as ex:
            print(pid, mcs, df.loc[mcs]["Sent"])
            skip_uids.append(mcs)
    
    for mcs in gt_dict[pid]["not_mcomp"]:
        try:
            mcomp_chunks_from_sent = extract_chunks_using_spacy_dp(df.loc[mcs]["Sent"])
            if mcomp_chunks_from_sent:
                final_chunks = mcomp_chunks_from_sent
            else:
                final_chunks = [df.loc[mcs]["Sent"]]
            
            roberta_vectors[pid][mcs] = []
            for single_chunk in final_chunks:
                vec = embed_text_using_roberta(single_chunk.strip()).mean(1).detach().numpy()
                roberta_vectors[pid][mcs].append(vec / norm(vec))
        except Exception as ex:
            print(pid, mcs, df.loc[mcs]["Sent"])
            skip_uids.append(mcs)

2019_HyVxPsC9tm 938 2
2019_HyVxPsC9tm 940 3
2018_HyHmGyZCZ 1425 2


In [27]:
mcomp_sentences = {}
not_mcomp_sentences = {}

for pid in gt_dict:
    for mcs in gt_dict[pid]["mcomp"]:
        if not mcs in skip_uids:
            mcomp_sentences[mcs] = pid
    for mcs in gt_dict[pid]["not_mcomp"]:
        if not mcs in skip_uids:
            not_mcomp_sentences[mcs] = pid
print(len(mcomp_sentences), len(not_mcomp_sentences))

117 1385


In [28]:
testdf = df.copy()

xtest = testdf.drop(columns=["PID", "Dec", "MComp", "Sent", "Cat", "SubCat"])
ytest = testdf.drop(columns=["PID", "Dec", "Sent", "Cat", "SubCat"])
# print(xtest.head())
# print(ytest.head())

for i in range(1, 769):
    xtest[i] = np.nan

for pid in gt_dict:
    for mcs in gt_dict[pid]["mcomp"]:
        if not mcs in skip_uids:
            xtest.iloc[mcs] = [mcs] + list(np.mean(roberta_vectors[pid][mcs], axis=0)[0])
        else:
            xtest.iloc[mcs] = [mcs] + list(np.zeros(768))
    for mcs in gt_dict[pid]["not_mcomp"]:
        if not mcs in skip_uids:
            xtest.iloc[mcs] = [mcs] + list(np.mean(roberta_vectors[pid][mcs], axis=0)[0])
        else:
            xtest.iloc[mcs] = [mcs] + list(np.zeros(768))
print(xtest.shape)

(1505, 769)


In [29]:
xtest.head(2)

,UID,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
0,0,0.042263,-0.009747,0.003180,-0.005164,-0.023304,0.002282,0.001261,0.041309,-0.005554,...,0.008538,-0.016800,0.000684,-0.038548,-0.027814,-0.013663,0.022429,0.018204,0.005023,0.015423
1,1,0.032411,0.013325,0.034656,0.016165,0.014038,-0.014843,-0.010618,0.001917,-0.010002,...,0.036052,-0.017437,-0.026445,-0.029300,-0.024152,-0.013721,0.035461,0.033084,-0.009003,0.026205


In [30]:
# xtrain = pd.DataFrame(columns=["UID"]+[str(x) for x in range(1,769)])
# ytrain = pd.DataFrame(columns=["UID", "MComp"])
xtlist = []
ytlist = []

for i in train_pool_uid_vecs:
    xtlist.append([i] + list(np.mean(train_pool_uid_vecs[i], axis=0)[0]))
    ytlist.append([i, mc_nmc_fake[i]])

xtrain = pd.DataFrame(xtlist)
ytrain = pd.DataFrame(ytlist)

xtrain = xtrain.rename(columns={0: 'UID'})
ytrain = ytrain.rename(columns={0: 'UID', 1: 'MComp'})

print(xtrain.shape)
print(ytrain.shape)

(296, 769)
(296, 2)


In [31]:
xtrain.head(2)

,UID,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
0,0,0.042346,-0.006763,-0.005576,-0.008125,0.008589,-0.025705,0.015792,0.026114,-0.011291,...,0.001838,0.000548,-0.022650,-0.049733,-0.013790,0.019725,-0.004193,0.030404,0.016235,0.017051
1,1,0.041915,0.013722,-0.018442,0.004288,-0.012606,-0.009582,-0.003896,0.033524,0.013773,...,-0.001821,0.005968,-0.013871,-0.024714,-0.029293,-0.006079,-0.003119,0.034447,0.012729,-0.010659


In [32]:
ytrain.head(2)

,UID,MComp
0,0,1
1,1,1


## ======================================================================

In [33]:
# Preprocessing & results----------------
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# nlp preprocessing
import spacy

# Models-------------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
import sklearn.gaussian_process.kernels as kls
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier

# for visualizing ---------------
from sklearn import tree
from six import StringIO 
from IPython.display import Image, display
import seaborn as sns
import graphviz
import matplotlib.pyplot as plt

# General purpose
import re
import pandas as pd
import pickle
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [46]:
# !pip3 install --user graphviz

In [34]:
np.random.seed(42)

In [35]:
clf_dict = {
    'DecisionTree': {"model": DecisionTreeClassifier(random_state=42), "params": {'max_depth': list(range(10, 250, 20))}},
    'RandomForest': {"model": RandomForestClassifier(random_state=42),
                     "params": {'n_estimators': list(range(5, 100, 5)), 'max_depth': list(range(10, 250, 20))}},
    'LogisticR_L1': {"model": LogisticRegression(random_state=42, max_iter=1000),
                     "params": {'penalty': ['l1'], 'solver': ['liblinear', 'saga']}},
    'LogisticR_L2': {"model": LogisticRegression(random_state=42, max_iter=1000),
                     "params": {'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}},
    'LogisticR': {"model": LogisticRegression(random_state=42, max_iter=1000),
                  "params": {'penalty': ['none'], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}},
    'RidgeClf': {"model": RidgeClassifier(max_iter=1000), "params": {}},
    'SVC_linear': {"model": SVC(random_state=42), "params": {'kernel': ['linear'], 
                                                             'C': [0.5, 1.0, 1.5, 2.0, 2.5]}},
    'SVC_poly': {"model": SVC(random_state=42),
                 "params": {'kernel': ['poly'], 'degree': [3, 4, 5], 'gamma': ['scale', 'auto'], 
                            'C': [0.5, 1.0, 1.5, 2.0, 2.5]}},
    'SVC_others': {"model": SVC(random_state=42), "params": {'kernel': ['rbf', 'sigmoid'], 
                                                             'gamma': ['scale', 'auto'], 
                                                             'C': [0.5, 1.0, 1.5, 2.0, 2.5]}},
    'GussianNB': {"model": GaussianNB(), "params": {}},
    'KNN': {"model": KNeighborsClassifier(), "params": {'n_neighbors': list(range(1, 20))}},
    'GaussianProcessClf': {"model": GaussianProcessClassifier(random_state=42, kernel=kls.RBF()), "params": {}},
    'Bagging_SVC': {"model": BaggingClassifier(random_state=42), "params": {'n_estimators': list(range(5, 100, 5)),
                                                                            'base_estimator': [SVC(kernel='linear'),
                                                                                               SVC(kernel='poly',
                                                                                                   degree=3,
                                                                                                   gamma='scale')]}},
    'BaggingDT': {"model": BaggingClassifier(random_state=42), "params": {'n_estimators': list(range(5, 100, 5)),
                                                                          'base_estimator': [
                                                                              DecisionTreeClassifier(random_state=42,
                                                                                                     max_depth=10),
                                                                              DecisionTreeClassifier(random_state=42,
                                                                                                     max_depth=50),
                                                                              DecisionTreeClassifier(random_state=42,
                                                                                                     max_depth=100)]}},
    'AdaBoost': {"model": AdaBoostClassifier(random_state=42), "params": {'n_estimators': list(range(5, 100, 5)),
                                                                          'base_estimator': [DecisionTreeClassifier(
                                                                                                 random_state=42,
                                                                                                 max_depth=10),
                                                                                             DecisionTreeClassifier(
                                                                                                 random_state=42,
                                                                                                 max_depth=50),
                                                                                             DecisionTreeClassifier(
                                                                                                 random_state=42,
                                                                                                 max_depth=100)]}},
    'ExtraTrees': {"model": ExtraTreesClassifier(random_state=42), "params": {'n_estimators': list(range(5, 105, 5)), 
                                                                              'max_depth': [10, 50, 100, 250, 400]}},
    'MLP_l1': {"model": MLPClassifier(random_state=42), "params": {'hidden_layer_sizes': [(x,) for x in 
                                                                                          range(50, 600, 100)], 
                                                                  'activation': ['logistic', 'tanh', 'relu'],
                                                                  'solver': ['adam', 'sgd'], 'early_stopping': 
                                                                   [True]}},
    'MLP_l2': {"model": MLPClassifier(random_state=42), "params": {'hidden_layer_sizes': [(x, y) for x in 
                                                                                          range(50, 600, 100) 
                                                                                          for y in range(50, 360, 100)], 
                                                                  'activation': ['logistic', 'tanh', 'relu'],
                                                                  'solver': ['adam', 'sgd'], 'early_stopping': 
                                                                                               [True]}},
#     'MLP_l3': {"model": MLPClassifier(random_state=42), "params": {'hidden_layer_sizes': [(x, y, z) for x in 
#                                                                                           range(50, 600, 100) 
#                                                                                           for y in range(50, 600, 100)
#                                                                                           for z in range(50, 360, 100)], 
#                                                                   'activation': ['logistic', 'tanh', 'relu'],
#                                                                   'solver': ['adam', 'sgd'], 'early_stopping': 
#                                                                                                [True]}},
    }


In [40]:
# model_results = pd.DataFrame()
# model_results['Train_Accuracy'] = None
# model_results['Test_Accuracy'] = None
# model_results['best_params'] = None

# # X_train_final = X_train_normalized.drop(columns=["ref_latest"])
# # X_test_normalized_remgsdata = X_test_normalized.drop(columns=["ref_latest"])
# # X_train_normalized_remgsdata = X_train_normalized.copy()
# # X_test_normalized_remgsdata = X_test_normalized.copy()

# xtrain_final = xtrain.drop(columns=["UID"])
# ytrain_final = ytrain.drop(columns=["UID"])

# xtest_final = xtest.drop(columns=["UID"])
# ytest_final = ytest.drop(columns=["UID"])


# best_clf_ours = None
# best_clf_val = 0

# for clf_name, clf in clf_dict.items():
#     classifier = GridSearchCV(clf['model'], clf['params'], n_jobs=5)
#     classifier.fit(xtrain_final, ytrain_final)
#     best_model = classifier.best_estimator_
#     print(clf_name, classifier.best_score_, classifier.best_params_)
    
#     y_predicted = best_model.predict(xtest_final)
#     test_acc = accuracy_score(ytest_final, y_predicted)
    
#     if test_acc > best_clf_val:
#         best_clf_val = test_acc
#         best_clf_ours = best_model
    
#     model_results.loc[clf_name, ['Train_Accuracy', 'Test_Accuracy', 'best_params']] = [classifier.best_score_, test_acc, classifier.best_params_]
#     clsr = classification_report(ytest_final, y_predicted)

# print("================================================================================")
# print(best_clf_ours)
# best_y_hat = best_clf_ours.predict(xtest_final)
# clsr = classification_report(ytest_final, best_y_hat)
# print(clsr)
# test_acc = accuracy_score(ytest_final, best_y_hat)
# print("Test acc:", test_acc )
# print("Weighted F1 score: ", f1_score(ytest_final, best_y_hat, average='weighted'))

In [36]:
# test_acc

In [37]:
# precision_recall_fscore_support(ytest_final, best_y_hat, average='macro')[2]

In [38]:
from sklearn.metrics import precision_recall_fscore_support

In [39]:
model_results = pd.DataFrame()
model_results['Train_Accuracy'] = None
model_results['Test_Accuracy'] = None
model_results['best_params'] = None

# X_train_final = X_train_normalized.drop(columns=["ref_latest"])
# X_test_normalized_remgsdata = X_test_normalized.drop(columns=["ref_latest"])
# X_train_normalized_remgsdata = X_train_normalized.copy()
# X_test_normalized_remgsdata = X_test_normalized.copy()

xtrain_final = xtrain.drop(columns=["UID"])
ytrain_final = ytrain.drop(columns=["UID"])

xtest_final = xtest.drop(columns=["UID"])
ytest_final = ytest.drop(columns=["UID"])


best_clf_ours = None
best_clf_val = 0

for clf_name, clf in clf_dict.items():
    classifier = GridSearchCV(clf['model'], clf['params'], n_jobs=5)
    classifier.fit(xtrain_final, ytrain_final)
    best_model = classifier.best_estimator_
    print(clf_name, classifier.best_score_, classifier.best_params_)
    
    y_predicted = best_model.predict(xtest_final)
    test_acc_macro = precision_recall_fscore_support(ytest_final, y_predicted, average='macro')[2]#accuracy_score(ytest_final, y_predicted)
    
    if test_acc_macro > best_clf_val:
        best_clf_val = test_acc_macro
        best_clf_ours = best_model
    
    model_results.loc[clf_name, ['Train_Accuracy', 'Test_Accuracy', 'best_params']] = [classifier.best_score_, test_acc_macro, classifier.best_params_]
    clsr = classification_report(ytest_final, y_predicted)

print("================================================================================")
print(best_clf_ours)
best_y_hat = best_clf_ours.predict(xtest_final)
clsr = classification_report(ytest_final, best_y_hat)
print(clsr)
test_acc = accuracy_score(ytest_final, best_y_hat)
print("Test acc:", test_acc )
print("Weighted F1 score: ", f1_score(ytest_final, best_y_hat, average='weighted'))

DecisionTree 0.831081081081081 {'max_depth': 10}
RandomForest 0.9121621621621622 {'max_depth': 10, 'n_estimators': 10}
LogisticR_L1 0.9121621621621622 {'penalty': 'l1', 'solver': 'liblinear'}
LogisticR_L2 0.9121621621621622 {'penalty': 'l2', 'solver': 'newton-cg'}
LogisticR 0.9222972972972973 {'penalty': 'none', 'solver': 'saga'}
RidgeClf 0.9121621621621622 {}
SVC_linear 0.9121621621621622 {'C': 0.5, 'kernel': 'linear'}
SVC_poly 0.918918918918919 {'C': 2.5, 'degree': 5, 'gamma': 'scale', 'kernel': 'poly'}
SVC_others 0.9121621621621622 {'C': 0.5, 'gamma': 'scale', 'kernel': 'rbf'}
GussianNB 0.8682432432432432 {}
KNN 0.9222972972972973 {'n_neighbors': 2}
GaussianProcessClf 0.9121621621621622 {}
Bagging_SVC 0.9121621621621622 {'base_estimator': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False), 'n

In [52]:
mean_sim_with_mcomp = defaultdict(dict)
mean_sim_with_not_mcomp = defaultdict(dict)
max_sim_with_mcomp = defaultdict(dict)
max_sim_with_not_mcomp = defaultdict(dict)

mean_at_k = ["1", "3", "5", "7", "10", "26"]


for sid in mcomp_sentences:
    
    # 1. With other training set mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["mcomp"]:
        for cvec2 in roberta_vectors[mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])
    
    
    # 2. With other training set non_mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["non_mcomp"]:
        for cvec2 in roberta_vectors[mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_not_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_not_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_not_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_not_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])




for sid in not_mcomp_sentences:
    
    # 1. With other training set mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["mcomp"]:
        for cvec2 in roberta_vectors[not_mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])
    
    
    # 2. With other training set non_mcomp sentences
    temp_list = []    
    for init_train_vec in train_pool_roberta_vecs["non_mcomp"]:
        for cvec2 in roberta_vectors[not_mcomp_sentences[sid]][sid]:
            temp_list.append(np.inner(init_train_vec, cvec2)[0][0])
        
    sorted_temp_list = sorted(temp_list, reverse=True)
    mean_sim_with_not_mcomp[sid]["mean"] = np.mean(sorted_temp_list)
    max_sim_with_not_mcomp[sid]["max"] = max(sorted_temp_list)
    for vv in mean_at_k:
        mean_sim_with_not_mcomp[sid]["mean_{}".format(vv)] = np.mean(sorted_temp_list[0:int(vv)])
#         max_sim_with_not_mcomp[sid]["max_{}".format(vv)] = max(sorted_temp_list[0:int(vv)])


In [53]:
def precision_at_k(sent_sim_dict, k=10, sim_type="max", mean_at=1):
    """mean_at can take values: 1, 3, 5, 7, 10, 26"""
    
    if sim_type == "max":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["max"], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        fp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
            else:
                fp += 1
        return tp/(tp+fp)
            
    elif sim_type == "mean":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["mean_"+str(mean_at)], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        fp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
            else:
                fp += 1
        return tp/(tp+fp)

In [54]:
def recall_at_k(sent_sim_dict, k=10, sim_type="max", mean_at=1):
    if sim_type == "max":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["max"], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
        return tp/min(k, 117)
            
    elif sim_type == "mean":
        sorted_sims = sorted(sent_sim_dict.items(), key=lambda x: x[1]["mean_"+str(mean_at)], reverse=True)
        top_k_sorted_sims = sorted_sims[0:k]
        tp = 0
        for i in top_k_sorted_sims:
            if df.loc[i[0]]["MComp"] == 1:
                tp += 1
        return tp/min(k, 117)

### Based on max similarity with meaningful group and removing entries more similar to non-meaningful group

In [56]:
# DIff of max similarity of meaningful and non-meaningful sentences

diff_max = {}
preserve_non_negative_max = {}

for x in max_sim_with_mcomp:
    diff_max_sim_for_x = max_sim_with_mcomp[x]["max"] - max_sim_with_not_mcomp[x]["max"]
    diff_max[x] = {"max": diff_max_sim_for_x}
    
    if diff_max_sim_for_x > 0:
        preserve_non_negative_max[x] = {"max": max_sim_with_mcomp[x]["max"]}
    else:
        print(x)
        preserve_non_negative_max[x] = {"max": 0}

39
17
28
76
108
112
124
184
155
157
159
235
236
271
287
312
308
401
449
519
573
627
615
672
673
657
669
671
714
707
739
806
808
809
810
870
830
931
933
909
926
972
950
994
1004
1047
1125
1162
1100
1102
1212
1243
1268
1281
1316
1318
1331
1333
1279
1347
1406
1390
1452
1504
1464
1497
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
18
19
21
22
23
24
25
26
29
31
32
33
37
38
40
41
42
43
44
45
46
48
49
51
52
53
54
55
56
58
59
60
61
62
64
65
66
67
70
71
72
73
74
75
77
78
79
80
81
82
83
85
88
89
90
91
92
93
94
95
96
97
98
99
100
128
101
102
103
104
105
106
107
109
111
114
115
116
117
118
119
120
121
122
123
125
127
130
131
132
133
134
135
136
137
138
139
141
142
143
144
145
146
147
148
149
150
151
152
153
154
156
158
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
183
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
256
257
258
259
260
261
262
263
264
265
266
268
269
222
2

In [58]:
print("P and R based on preserving non-neg max sim with initial pool of \n meaningful & non meaningful sents:")

patk = [1, 5, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
res_table = [["Precision at:"] + patk, ["Val"]]

for k in patk:
    v1 = precision_at_k(preserve_non_negative_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1, 2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



ratk = [1, 5, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
res_table = [["Recall at:"] + ratk, ["Val"]]

for k in ratk:
    v1 = recall_at_k(preserve_non_negative_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1,2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))

P and R based on preserving non-neg max sim with initial pool of 
 meaningful & non meaningful sents:


Precision at:,1,5,10,20,50,100,117,150,200,300,400,500,600
Val,1,1,1,0.6,0.52,0.38,0.36,0.43,0.57,0.39,0.29,0.23,0.2


Recall at:,1,5,10,20,50,100,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,1,1,1,0.6,0.52,0.38,0.36,0.56,0.98,1,1,1,1,1,1,1,1,1,1


### Based on diff of max similarity with initial pool

In [59]:
print("P and R based on diff of max sim with initial pool of \n meaningful & non meaningful sents:")

patk = [1, 10, 20, 50, 100, 117, 150, 200, 300, 400, 500, 600]
res_table = [["Precision at:"] + patk, ["Val"]]

for k in patk:
    v1 = precision_at_k(diff_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1, 2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))



ratk = [117, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1388]
res_table = [["Recall at:"] + ratk, ["Val"]]

for k in ratk:
    v1 = recall_at_k(diff_max, k) #round(sum(i > 0 for i in diff12[val])/len(diff12[val]), 2)
    res_table[1].append(round(v1,2))

display(HTML(tabulate.tabulate(res_table, tablefmt='html')))

P and R based on diff of max sim with initial pool of 
 meaningful & non meaningful sents:


Precision at:,1,10,20,50,100,117,150,200,300,400,500,600
Val,1,0.6,0.5,0.44,0.41,0.37,0.37,0.32,0.25,0.21,0.18,0.15


Recall at:,117,150,200,300,400,500,600,700,800,900,1000,1200,1388
Val,0.37,0.47,0.54,0.65,0.72,0.77,0.79,0.85,0.85,0.89,0.92,0.96,0.99
